# 01. Exploratory analysis of building footprints

Цель:
- загрузить контуры зданий
- привести геометрию в метрическую систему координат
- вычислить площадь крыш
- сохранить промежуточный датасет


In [8]:
import geopandas as gpd
from pathlib import Path
import pandas as pd
from shapely.geometry import shape

In [4]:
PROJECT_ROOT = Path("/content/drive/MyDrive/solar_potential_project")

DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_INTERIM = PROJECT_ROOT / "data" / "interim"

DATA_RAW, DATA_INTERIM

(PosixPath('/content/drive/MyDrive/solar_potential_project/data/raw'),
 PosixPath('/content/drive/MyDrive/solar_potential_project/data/interim'))

загрузка фрагмента открытого датасета Global ML Building Footprints для территории Эстонии

In [10]:
path = Path("/content/drive/MyDrive/solar_potential_project/data/raw/part-00189-5cf70943-9c5f-4fc6-94fb-43ce5feefa56.c000.csv")

df = pd.read_json(path, lines=True)
df.head()

,type,properties,geometry
0,Feature,"{'height': -1.0, 'confidence': -1.0}","{'type': 'Polygon', 'coordinates': [[[22.19814..."
1,Feature,"{'height': -1.0, 'confidence': -1.0}","{'type': 'Polygon', 'coordinates': [[[22.05148..."
2,Feature,"{'height': -1.0, 'confidence': -1.0}","{'type': 'Polygon', 'coordinates': [[[22.27938..."
3,Feature,"{'height': -1.0, 'confidence': -1.0}","{'type': 'Polygon', 'coordinates': [[[22.25336..."
4,Feature,"{'height': -1.0, 'confidence': -1.0}","{'type': 'Polygon', 'coordinates': [[[22.30554..."


In [11]:
props = pd.json_normalize(df["properties"])
geoms = df["geometry"].apply(shape)

gdf = gpd.GeoDataFrame(props, geometry=geoms, crs="EPSG:4326")
gdf.head()

,height,confidence,geometry
0,-1.0,-1.0,"POLYGON ((22.19815 58.91611, 22.19816 58.91614..."
1,-1.0,-1.0,"POLYGON ((22.05148 58.94249, 22.05145 58.94252..."
2,-1.0,-1.0,"POLYGON ((22.27939 58.89897, 22.27931 58.89897..."
3,-1.0,-1.0,"POLYGON ((22.25337 58.88799, 22.25322 58.88799..."
4,-1.0,-1.0,"POLYGON ((22.30554 58.90718, 22.3054 58.90717,..."


In [12]:
gdf.columns, len(gdf), gdf.crs

(Index(['height', 'confidence', 'geometry'], dtype='object'),
 2132,
 <Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

перевод площади из градусов в квадратные метры

In [13]:
gdf_utm = gdf.to_crs(gdf.estimate_utm_crs())
gdf_utm["area_m2"] = gdf_utm.geometry.area

gdf_utm[["area_m2"]].describe()

,area_m2
count,2132.000000
mean,78.173866
std,230.335725
min,3.463815
25%,18.916592
50%,42.697945
75%,91.738530
max,8711.480192


In [ ]:
out = Path("/content/drive/MyDrive/solar_potential_project/data/interim/estonia_tile_with_area.geojson")
gdf_utm.to_file(out, driver="GeoJSON")

In [15]:
gdf_utm.sample(min(500, len(gdf_utm))).explore()